In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
from transformers.trainer_utils import get_last_checkpoint

latest = get_last_checkpoint("fpf-1b-32k")
latest

'fpf-1b-32k/checkpoint-17900'

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [4]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/llama-7b-hf-32768-fpf', trust_remote_code=True)

In [5]:
model = AutoModelForCausalLM.from_pretrained(latest, torch_dtype=torch.float16)

/home/ubuntu/.local/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/ubuntu/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
[2023-09-19 12:53:21,419] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [6]:
model.config

LlamaConfig {
  "_name_or_path": "fpf-1b-32k/checkpoint-17900",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 32768,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 4,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.31.0",
  "use_cache": true,
  "vocab_size": 32000
}

In [7]:
model.save_pretrained('./out-1b', safe_serialization=True)
tokenizer.save_pretrained('./out-1b', safe_serialization=True)

('./out-1b/tokenizer_config.json',
 './out-1b/special_tokens_map.json',
 './out-1b/tokenizer.json')

In [8]:
!ls -lh out-1b

total 2.0G
-rw-r--r-- 1 ubuntu ubuntu  628 Sep 19 12:53 config.json
-rw-r--r-- 1 ubuntu ubuntu  132 Sep 19 12:53 generation_config.json
-rw-r--r-- 1 ubuntu ubuntu 2.0G Sep 19 12:53 model.safetensors
-rw-r--r-- 1 ubuntu ubuntu  414 Sep 19 12:53 special_tokens_map.json
-rw-r--r-- 1 ubuntu ubuntu 1.8M Sep 19 12:53 tokenizer.json
-rw-r--r-- 1 ubuntu ubuntu  725 Sep 19 12:53 tokenizer_config.json


In [9]:
!sudo apt install sshpass -y
!ssh-keygen -f "/home/ubuntu/.ssh/known_hosts" -R "jupyter-serve"
!echo 'put -r out-1b .' | sshpass -p ubuntu123 sftp -o StrictHostKeyChecking=no ubuntu@jupyter-serve

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
sshpass is already the newest version (1.09-1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
W: Target Packages (main/binary-amd64/Packages) is configured multiple times in /etc/apt/sources.list.d/google-chrome.list:3 and /etc/apt/sources.list.d/google.list:1
W: Target Packages (main/binary-all/Packages) is configured multiple times in /etc/apt/sources.list.d/google-chrome.list:3 and /etc/apt/sources.list.d/google.list:1
# Host jupyter-serve found: line 5
/home/ubuntu/.ssh/known_hosts updated.
Original contents retained as /home/ubuntu/.ssh/known_hosts.old
Connected to jupyter-serve.
sftp> put -r out-1b .
Uploading out-1b/ to /home/ubuntu/./out-1b
Entering out-1b/


In [10]:
model.push_to_hub('llama-1b-hf-32768-fpf', organization='mesolitica', safe_serialization=True)

/home/ubuntu/.local/lib/python3.10/site-packages/transformers/utils/hub.py:652: UserWarning: The `organization` argument is deprecated and will be removed in v5 of Transformers. Set your organization directly in the `repo_id` passed instead (`repo_id={organization}/{model_id}`).
  warnings.warn(


model.safetensors:   0%|          | 0.00/2.14G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/llama-1b-hf-32768-fpf/commit/f4adf91594daa73a1cf057d4dbfd3b8381b01113', commit_message='Upload LlamaForCausalLM', commit_description='', oid='f4adf91594daa73a1cf057d4dbfd3b8381b01113', pr_url=None, pr_revision=None, pr_num=None)

In [11]:
tokenizer.push_to_hub('llama-1b-hf-32768-fpf', organization='mesolitica')

CommitInfo(commit_url='https://huggingface.co/mesolitica/llama-1b-hf-32768-fpf/commit/5d1e1b67a6dfa1794624d7248004c14f95466687', commit_message='Upload tokenizer', commit_description='', oid='5d1e1b67a6dfa1794624d7248004c14f95466687', pr_url=None, pr_revision=None, pr_num=None)